There are many parameters we can optmize, for instance, the learning rate, optimizing function, number and types of layers, neurons per layer

Let's tweak the number of layers and nodes per layer for now

In [1]:
import os
import time
import pickle

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

In [2]:
# define variables
home = os.getenv('HOME')
datadir = os.path.join(home, 'Pictures', 'kagglecatsanddogs_3367a', 'PetImages')

# tensorflow variables
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.33)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
# load data
with open(os.path.join(datadir, 'X.pkl'), 'rb') as file:
    X = pickle.load(file)
    
with open(os.path.join(datadir, 'y.pkl'), 'rb') as file:
    y = pickle.load(file)

In [4]:
# normalize the data
X = X/255

In [5]:
X.shape

(24946, 50, 50, 1)

In [6]:
# parameter grid
dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

In [7]:
# iterate over all parameters in greedy grid search
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            name = ['{}-conv-{}-nodes-{}-dense-{}'.format(conv_layer, layer_size, conv_layer, int(time.time()))]
            tensorboard = TensorBoard(log_dir='logs/{}'.format(name))
            print(name)

            # declaring a model
            model = Sequential()

            # convolution layer with 64 neuros
            model.add(Conv2D(filters=(layer_size), kernel_size=(3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))
            
            for layer in range(conv_layer - 1):
                # convolution layer with 64 neurons
                model.add(Conv2D(filters=(layer_size), kernel_size=(3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))
            
            # flatten the output
            model.add(Flatten())
                
            for layer in range(dense_layer):
                model.add(Dense(units=layer_size))
                model.add(Activation('relu'))

            # dense layer with 64 neurons
            # model.add(Dense(units=64))
            # model.add(Activation('relu'))

            # output layer with sigmoid activation
            model.add(Dense(units=1, activation='sigmoid'))
            
            # compiling the model
            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
            
            # fitting the model
            model.fit(x=X, y=y, batch_size=32, epochs=20, validation_split=0.2, callbacks=[tensorboard])
            
# save the model
model.save(os.path.join(home, 'intro-dl', 'models', '64x3-cnn.model'))

['3-conv-64-nodes-3-dense-1548621921']
Train on 19956 samples, validate on 4990 samples
Epoch 1/20
19956/19956 [==============================] - 4s 198us/step - loss: 0.6412 - acc: 0.6230 - val_loss: 0.5807 - val_acc: 0.7012
Epoch 2/20
19956/19956 [==============================] - 3s 156us/step - loss: 0.5485 - acc: 0.7233 - val_loss: 0.5141 - val_acc: 0.7513
Epoch 3/20
19956/19956 [==============================] - 3s 158us/step - loss: 0.4883 - acc: 0.7652 - val_loss: 0.4881 - val_acc: 0.7581
Epoch 4/20
19956/19956 [==============================] - 3s 155us/step - loss: 0.4496 - acc: 0.7934 - val_loss: 0.4842 - val_acc: 0.7788
Epoch 5/20
19956/19956 [==============================] - 3s 167us/step - loss: 0.4177 - acc: 0.8087 - val_loss: 0.4445 - val_acc: 0.7962
Epoch 6/20
19956/19956 [==============================] - 3s 169us/step - loss: 0.3829 - acc: 0.8287 - val_loss: 0.4203 - val_acc: 0.8084
Epoch 7/20
19956/19956 [==============================] - 3s 161us/step - loss: 0.35

# testing on new images

In [8]:
# downloading test images
!wget -P ~/Pictures/kagglecatsanddogs_3367a/PetImages/test/ https://pythonprogramming.net/static/images/machine-learning/dog.jpg
!wget -P ~/Pictures/kagglecatsanddogs_3367a/PetImages/test/ https://pythonprogramming.net/static/images/machine-learning/cat.jpg

--2019-01-27 18:46:26--  https://pythonprogramming.net/static/images/machine-learning/dog.jpg
Resolving pythonprogramming.net (pythonprogramming.net)... 104.237.143.20, 2600:3c00::f03c:91ff:fe84:176d
Connecting to pythonprogramming.net (pythonprogramming.net)|104.237.143.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60603 (59K) [image/jpeg]
Saving to: ‘/home/jayme/Pictures/kagglecatsanddogs_3367a/PetImages/test/dog.jpg’

dog.jpg             100%[===================>]  59,18K   156KB/s    in 0,4s    

2019-01-27 18:46:27 (156 KB/s) - ‘/home/jayme/Pictures/kagglecatsanddogs_3367a/PetImages/test/dog.jpg’ saved [60603/60603]

--2019-01-27 18:46:27--  https://pythonprogramming.net/static/images/machine-learning/cat.jpg
Resolving pythonprogramming.net (pythonprogramming.net)... 104.237.143.20, 2600:3c00::f03c:91ff:fe84:176d
Connecting to pythonprogramming.net (pythonprogramming.net)|104.237.143.20|:443... connected.
HTTP request sent, awaiting response... 200 O

In [9]:
import cv2
import tensorflow as tf

In [10]:
categories = ['Cat', 'Dog']

In [11]:
def prepare(file_path):
    image_size = 50
    image_array = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(image_array, (image_size, image_size))
    return new_array.reshape(-1, image_size, image_size, 1)

In [12]:
# loading the model
model = tf.keras.models.load_model(os.path.join(home, 'intro-dl', 'models', '64x3-cnn.model'))

In [13]:
# make a prediction
prediction = model.predict([prepare(file_path=os.path.join(datadir, 'test', 'dog.jpg'))])
# pretty print the prediction
print(categories[(int(prediction[0][0]))])

Dog


In [14]:
# make a prediction
prediction = model.predict([prepare(file_path=os.path.join(datadir, 'test', 'cat.jpg'))])
# pretty print the prediction
print(categories[(int(prediction[0][0]))])

Cat
